In [1]:
import pandas as pd
import numpy as np

In [4]:
dataset = pd.read_excel("./data/Анализ финансирования на 23.06.2022.xls", sheet_name="ОБ   ")
dataset = pd.read_excel("./data/Анализ финансирования на 04.08.2022.xls", sheet_name="ОБ")
dataset.head(20)

,АНАЛИЗ ФИНАНСИРОВАНИЯ за 2022 г.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,по состоянию на 04 августа 2022 г.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,тыс.руб.
3,№№ п/п,NaN,Наименование статей расходов,NaN,NaN,Бюджетные ассигнования на 2022 год,Лимиты бюджетных обязательств на 2022 год,Кассовый план,NaN,Профинанси-ровано,% финансирования,NaN,NaN,Выставленные заявки на оплату расходов
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Выделенный Облфином,Неиспользовано по состоянию \nна 04 августа 20...,NaN,от бюджетных ассигований на 2022 г.,от лимитов бюджетных обязательств на 2022 г.,от кассового плана,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Итого из областного бюджета,NaN,NaN,16706364.66,16633208.36,11863431.59,2026689.56,9815612.07,58.753728,59.012139,82.738388,21129.96
7,1,NaN,Обеспечение деятельности подведомственных учре...,NaN,NaN,5626499.56,5599343.93,3983330,928209.37,3051781.18,54.239428,54.502478,76.613818,3339.45
8,NaN,NaN,"Мероприятия, связанные с профилактикой и устра...",NaN,NaN,42966.91,35945.18,35945.17,67.13,35878.04,83.50156,99.813216,99.813243,0
9,2,NaN,Социальное обеспечение - всего\n в том ...,NaN,NaN,10970142.32,10925669.39,7792317.36,1082801.55,6691750.3,60.999667,61.247966,85.876255,17765.51


In [5]:
def row_stringify(row):
	row = map(str, list(row.values))
	str_row = "".join(row)
	return str_row

def coords(dataset, name):
	row_index = 0
	for i in range(len(dataset)):
		row = dataset.iloc[i]
		row_str = row_stringify(row)
		if name in row_str:
			row_index = i
			break
	col_index = 0
	for j in range(len(dataset.columns)):
		col = dataset.iloc[row_index, j]
		if name in str(col):
			col_index = j
			break
	
	return np.array([row_index, col_index], dtype=np.int64)

In [6]:
start_coords_y, start_coords_x = coords(dataset, "Наименование статей расходов")

In [7]:
dataset.fillna(0, inplace=True)

In [8]:
new_dataset_columns = [
	'Дата',
 	'Наименование статей расходов',
 	'Категория',
 	'Наименование подстатей расходов',
 	'Бюджетные ассигнования на 2022 год ',
 	'Лимиты бюджетных обязательств на 2022 год',
 	'КП Выделенный Облфином',
 	'КП Неиспользовано по состоянию на 27 января 2022г.(не выставлено заявок)',
 	'Профинансировано',
 	'% от бюджетных ассигований на 2022 г.',
 	'% от лимитов бюджетных обязательств на 2022 г.',
 	'% от кассового плана',
 	'Выставленные заявки на оплату расходов ',
 	'Единица измерения'
]

flat_dataset = {name: [] for name in new_dataset_columns}
name = ""
subname = ""
empty_to_zero = lambda x: x if x != "" else 0

for i in range(start_coords_y+6, len(dataset)):
	# print(i)
	row = dataset.iloc[i]
	str_row = row_stringify(row=row)
	
	if not "COVID" in str_row:
		name_1 = dataset.iloc[i, 2].replace("\n", "")
		if "-" in name_1:
			name_1 = name_1[:name_1.index("-")]
		num = dataset.iloc[i, 0]
		if str(num).replace(".", "").isdigit():
			if "." in str(num):
				subname = name_1
			else:
				name = name_1
				subname = name
		flat_dataset['Дата'].append("")
		flat_dataset['Наименование статей расходов'].append(name)
		flat_dataset['Категория'].append(subname)
		flat_dataset['Наименование подстатей расходов'].append(name_1)
		flat_dataset['Бюджетные ассигнования на 2022 год '].append(empty_to_zero(dataset.iloc[i, 5]))
		flat_dataset['Лимиты бюджетных обязательств на 2022 год'].append(empty_to_zero(dataset.iloc[i, 6]))
		flat_dataset['КП Выделенный Облфином'].append(empty_to_zero(dataset.iloc[i, 7]))
		flat_dataset['КП Неиспользовано по состоянию на 27 января 2022г.(не выставлено заявок)'].append(empty_to_zero(dataset.iloc[i, 8]))
		flat_dataset['Профинансировано'].append(empty_to_zero(dataset.iloc[i, 9]))
		flat_dataset['% от бюджетных ассигований на 2022 г.'].append(empty_to_zero(dataset.iloc[i, 10]))
		flat_dataset['% от лимитов бюджетных обязательств на 2022 г.'].append(empty_to_zero(dataset.iloc[i, 11]))
		flat_dataset['% от кассового плана'].append(empty_to_zero(dataset.iloc[i, 12]))
		flat_dataset['Выставленные заявки на оплату расходов '].append(empty_to_zero(dataset.iloc[i, 13]))
		flat_dataset['Единица измерения'].append("тыс. руб")
		# print(dataset.iloc[i, 2])
	
	if "Аппарат комитета" in str_row:
		break

pd.DataFrame(data=flat_dataset).to_excel("./test.xlsx")


In [40]:
dataset

,АНАЛИЗ ФИНАНСИРОВАНИЯ за 2022 г.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,по состоянию на 23 июня 2022 г.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,тыс.руб.
3,№№ п/п,NaN,Наименование статей расходов,NaN,NaN,Бюджетные ассигнования на 2022 год,Лимиты бюджетных обязательств на 2022 год,Кассовый план,NaN,Профинанси-ровано,% финансирования,NaN,NaN,Выставленные заявки на оплату расходов
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Выделенный Облфином,Неиспользовано по состоянию \nна 23 июня 2022г...,NaN,от бюджетных ассигований на 2022 г.,от лимитов бюджетных обязательств на 2022 г.,от кассового плана,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,2.36,NaN,"Подпрограмма ""Формирование доступной среды жиз...",NaN,з,4122.55,3470.22,1753.41,70.28,1683.13,40.827401,48.502112,95.99181,0
87,2.37,NaN,Государственная программа Волгоградской област...,ч,NaN,23399.16,22631.63,2451.51,13.6,2428.91,10.38033,10.732369,99.078119,9
88,3,NaN,Аппарат комитета,NaN,NaN,101121.28,99602.87,58969.33,8213.65,50755.68,50.192877,50.95805,86.071319,0
89,4,NaN,Достижение показателей деятельности органов ис...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
